In [1]:
import sys
sys.path.append("../Modules/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np

from configobj import ConfigObj
from configobj import flatten_errors
from validate import Validator
    
import logging

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Subject number

In [3]:
subNum = 0;
trIdx = 0;
# gazeConfidenceThreshold = 0.6;

### Load dataframe
Remember to set loadParsedData, loadProcessedData.

In [4]:
dataFolderList = [];
[dataFolderList.append(name) for name in os.listdir("../Data/") if name[0] is not '.'];

dataParentFolder = "../Data/" + dataFolderList[subNum];

dataSubFolderList = [];
[dataSubFolderList.append(name) for name in os.listdir("../Data/" + dataParentFolder) if name[0] is not '.'];

dataFolder = dataParentFolder + '/' + dataSubFolderList[0] + '/' 

gazeDataFolderList = [];
[gazeDataFolderList.append(name) for name in os.listdir(dataFolder + 'PupilData') if name[0] is not '.'];

pupilSessionFolder = '/' + gazeDataFolderList[0] +  '/' 

gazeDataFolder = dataFolder + 'PupilData' + pupilSessionFolder

### Iterate through the rows of trial_results.csv

In [5]:
allTrialResultsDf = pd.read_csv( dataFolder + 'trial_results.csv')

In [6]:
trDataFiles = [i for i in allTrialResultsDf.columns.to_list() if '_filename' in i]

In [7]:
logger.info('Processing subject: ' + allTrialResultsDf.iloc[trIdx]['ppid'] + ' t = ' + str(allTrialResultsDf.iloc[trIdx]['trial_num']))

INFO___main__-<module>(): - Processing subject: P_201218121321 t = 1


Rename trial data columns

In [8]:
# allTrialResultsDf.rename(columns={"session_num":"sessionNumber","trial_num":"trialNumber",
#         "block_num":"blockNumber","trial_num_in_block":"trialNumberInBlock",
#         "start_time":"startTime","end_time":"endTime"},inplace=True)

In [9]:
allTrialResultsDf.keys()

Index(['directory', 'experiment', 'ppid', 'session_num', 'trial_num',
       'block_num', 'trial_num_in_block', 'start_time', 'end_time',
       'isCaughtQ', 'contactLocOnPaddle_x', 'contactLocOnPaddle_y',
       'contactLocOnPaddle_z', 'gazeTargetPositionInHead_x',
       'gazeTargetPositionInHead_y', 'gazeTargetPositionInHead_z', 'trialType',
       'ballInitialPos_x', 'ballInitialPos_y', 'ballInitialPos_z',
       'ballInitialVel_x', 'ballInitialVel_y', 'ballInitialVel_z',
       'ballFinalPos_x', 'ballFinalPos_y', 'ballFinalPos_z', 'maxReach',
       'noExpansionLastXSeconds', 'eyeHeight', 'isLeftHanded', 'timeOfContact',
       'contactLocinWorld_x', 'contactLocinWorld_y', 'contactLocinWorld_z',
       'camera_movement_filename', 'pupil_pupilTimeStamp_filename',
       'paddle_movement_filename',
       'etassessment_calibrationAssessment_filename'],
      dtype='object')

In [10]:
trialResults = allTrialResultsDf.loc[trIdx]
trNum = int(trialResults['trial_num'])

In [11]:
## Import view data and rename some columns
dataFileName = trialResults['camera_movement_filename']
viewData = pd.read_csv( dataFolder + dataFileName)
viewData = viewData.rename(columns={"time": "frameTime"})

## Import pupil timestamp data (recorded within Unity)
dataFileName = trialResults['pupil_pupilTimeStamp_filename']
pupilTimestampData = pd.read_csv( dataFolder + dataFileName)
pupilTimestampData = pupilTimestampData.rename(columns={"time": "frameTime"})

In [12]:
## Import view data and rename some columns
dataFileName = trialResults['camera_movement_filename']
viewData = pd.read_csv( dataFolder + dataFileName)
viewData = viewData.rename(columns={"time": "frameTime"})

## Import pupil timestamp data (recorded within Unity)
dataFileName = trialResults['pupil_pupilTimeStamp_filename']
pupilTimestampData = pd.read_csv( dataFolder + dataFileName)
pupilTimestampData = pupilTimestampData.rename(columns={"time": "frameTime"})

## Import gaze direction data
gazeDataFolderList = []
[gazeDataFolderList.append(name) for name in os.listdir(dataFolder + 'PupilData') if name[0] is not '.']

[None]

In [13]:
# Import gaze data exported from pupil player
pupilExportsFolder = []
[pupilExportsFolder.append(name) for name in os.listdir(gazeDataFolder + '/Exports') if name[0] is not '.']

# Defaults to the most recent pupil export folder (highest number)
gazePositionsDF = pd.read_csv( gazeDataFolder + '/Exports/' + pupilExportsFolder[-1] + '/gaze_positions.csv' )
gazePositionsDF.head()

,gaze_timestamp,world_index,confidence,norm_pos_x,norm_pos_y,base_data,gaze_point_3d_x,gaze_point_3d_y,gaze_point_3d_z,eye_center0_3d_x,eye_center0_3d_y,eye_center0_3d_z,gaze_normal0_x,gaze_normal0_y,gaze_normal0_z,eye_center1_3d_x,eye_center1_3d_y,eye_center1_3d_z,gaze_normal1_x,gaze_normal1_y,gaze_normal1_z
0,701735.565136,1,0.00000,0.477686,0.160888,701735.5651359999-0,-4.554664,51.913864,132.577788,4.554664,-51.913864,-132.577788,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,701735.582292,1,0.00000,0.477686,0.160888,701735.582292-0,-4.554664,51.913864,132.577788,4.554664,-51.913864,-132.577788,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,701735.591153,1,0.00000,0.477686,0.160888,701735.591153-0,-4.554664,51.913864,132.577788,4.554664,-51.913864,-132.577788,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,701735.599210,1,0.00000,0.477686,0.160888,701735.59921-0,-4.554664,51.913864,132.577788,4.554664,-51.913864,-132.577788,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
4,701735.607214,1,0.73569,0.623874,0.304769,701735.607214-0,66.045043,78.067645,346.299796,4.554664,-51.913864,-132.577788,0.122981,0.259963,0.957755,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
gazePositionsDF = gazePositionsDF.rename(columns={"gaze_timestamp": "pupilTimestamp"})

# Sort, because the left/right eye data is written asynchronously, and this means timestamps may not be monotonically increasing.
gazePositionsDF.sort_values(by='pupilTimestamp',inplace=True)

In [15]:
##################################################################################################
##################################################################################################
## Create rawTrialData and merge with view data

rawTrialData = pupilTimestampData;
rawTrialData['trialNumber'] = trialResults['trial_num']
rawTrialData['blockNumber'] = trialResults['block_num']
rawTrialData = pd.merge(rawTrialData, viewData, on ='frameTime',validate= 'one_to_many')

In [16]:
# ballData = []
# paddleData = []

# if(trialResults['trialType'] == 'interception'):

#     logger.info('Trial type: Interception')
    
#     ## Import ball data and rename some columns
#     dataFileName = trialResults['ball_movement_filename']
#     ballData = pd.read_csv( dataFolder + dataFileName)
#     ballData = ballData.rename(columns={"time": "frameTime"})
#     ballData.rename(columns={"pos_x": "ballPos_x", "pos_y": "ballPos_y","pos_z": "ballPos_z"},inplace=True)
#     ballData.rename(columns={"rot_x": "ballRot_x", "rot_y": "ballRot_y","rot_z": "ballRot_z"},inplace=True)

#     ## Import paddle data and rename some columns
#     dataFileName = trialResults['paddle_movement_filename']
#     paddleData = pd.read_csv( dataFolder + dataFileName)
#     paddleData = paddleData.rename(columns={"time": "frameTime"})

#     ## Merge view and ball data into rawTrialData
#     rawTrialData = pd.merge(rawTrialData, ballData, on ='frameTime',validate= 'one_to_many')
#     rawTrialData = pd.merge(rawTrialData, paddleData, on ='frameTime',validate= 'one_to_many')

# if(trialResults['trialType'] == 'CalibrationAssessment'):

#     logger.info('Trial type: CalibrationAssessment')
    
#     ## Import ball data and rename some columns
#     dataFileName = trialResults['etassessment_calibrationAssessment_filename']
#     assessmentData = pd.read_csv( dataFolder + dataFileName)
#     assessmentData = assessmentData.rename(columns={"time": "frameTime"})

#     rawTrialData = pd.merge(rawTrialData, assessmentData, on ='frameTime',validate= 'one_to_many')

In [17]:
ballData = []
paddleData = []

if(trialResults['trialType'] == 'interception'):

    ## Import ball data and rename some columns
    dataFileName = trialResults['ball_movement_filename']
    ballData = pd.read_csv( dataFolder + dataFileName)
    ballData = ballData.rename(columns={"time": "frameTime"})
    ballData.rename(columns={"pos_x": "ballPos_x", "pos_y": "ballPos_y","pos_z": "ballPos_z"},inplace=True)
    ballData.rename(columns={"rot_x": "ballRot_x", "rot_y": "ballRot_y","rot_z": "ballRot_z"},inplace=True)

    ## Import paddle data and rename some columns
    dataFileName = trialResults['paddle_movement_filename']
    paddleData = pd.read_csv( dataFolder + dataFileName)
    paddleData = paddleData.rename(columns={"time": "frameTime"})

    ## Merge view and ball data into rawTrialData
    rawTrialData = pd.merge(rawTrialData, ballData, on ='frameTime',validate= 'one_to_many')
    rawTrialData = pd.merge(rawTrialData, paddleData, on ='frameTime',validate= 'one_to_many')

if(trialResults['trialType'] == 'CalibrationAssessment'):
    
    ## Import ball data and rename some columns
    dataFileName = trialResults['etassessment_calibrationAssessment_filename']
    assessmentData = pd.read_csv( dataFolder + dataFileName)
    assessmentData = assessmentData.rename(columns={"isHeadFixed_x": "isHeadFixed"})
    
    newKeys = ['frameTime']
    [newKeys.append(key[13:]) for key in assessmentData.keys()[1:]] # Fix a silly mistake I made when naming columns
    assessmentData.columns = newKeys

    rawTrialData = pd.merge(rawTrialData, assessmentData, on ='frameTime',validate= 'one_to_many')


In [18]:
gb = rawTrialData.groupby('pupilTimestamp')

In [19]:
################################################################################
## Pupil gaze direction data is for the whole experiment while time stamps are recorded only during an ongoing trial.
## Find the slices of gaze dir data that map onto the trial timestamps, and concatenate them.

## Group trial data by blocks and trials
gbBlTr = rawTrialData.groupby(['blockNumber','trialNumber'])
tr = gbBlTr.get_group( list(gbBlTr.groups.keys())[0])

firstTS = tr.head(1)['pupilTimestamp']
lastTS = tr.tail(1)['pupilTimestamp']
firstIdx = list(map(lambda i: i> float(firstTS), gazePositionsDF['pupilTimestamp'])).index(True)
lastIdx = list(map(lambda i: i> float(lastTS), gazePositionsDF['pupilTimestamp'])).index(True)
rawGazeData = gazePositionsDF.loc[firstIdx:lastIdx]

In [20]:
rawGazeData

,pupilTimestamp,world_index,confidence,norm_pos_x,norm_pos_y,base_data,gaze_point_3d_x,gaze_point_3d_y,gaze_point_3d_z,eye_center0_3d_x,eye_center0_3d_y,eye_center0_3d_z,gaze_normal0_x,gaze_normal0_y,gaze_normal0_z,eye_center1_3d_x,eye_center1_3d_y,eye_center1_3d_z,gaze_normal1_x,gaze_normal1_y,gaze_normal1_z
10424,701824.835411,2612,0.007803,-0.336810,0.443441,701824.835411-0,-313.867594,15.910394,243.619161,3.974694,-54.166874,-135.940395,-0.635685,0.140155,0.759119,NaN,NaN,NaN,NaN,NaN,NaN
10425,701824.843296,2612,0.478732,0.125579,0.804740,701824.843296-0,-186.034658,-113.559815,322.719528,3.974694,-54.166874,-135.940395,-0.380019,-0.118786,0.917320,NaN,NaN,NaN,NaN,NaN,NaN
10426,701824.851192,2612,0.001554,0.070826,0.643217,701824.851192-0,-209.063097,-52.323880,316.399635,3.974694,-54.166874,-135.940395,-0.426076,0.003686,0.904680,NaN,NaN,NaN,NaN,NaN,NaN
10427,701824.859296,2612,0.003006,-0.117173,0.457648,701824.859296-0,-265.684695,13.674031,279.609151,3.974694,-54.166874,-135.940395,-0.539319,0.135682,0.831099,NaN,NaN,NaN,NaN,NaN,NaN
10428,701824.868456,2613,0.000261,0.123428,0.805588,701824.868456-0,-186.884977,-113.742650,322.342610,3.974694,-54.166874,-135.940395,-0.381719,-0.119152,0.916566,NaN,NaN,NaN,NaN,NaN,NaN
10429,701824.876640,2613,0.004712,-0.048342,0.517820,701824.87664-0,-247.704435,-6.037488,293.409320,3.974694,-54.166874,-135.940395,-0.503358,0.096259,0.858699,NaN,NaN,NaN,NaN,NaN,NaN
10430,701824.887225,2613,0.005916,-0.072120,0.352065,701824.887225-0,-249.172849,48.321965,282.882191,3.974694,-54.166874,-135.940395,-0.506295,0.204978,0.837645,NaN,NaN,NaN,NaN,NaN,NaN
10431,701824.896308,2614,0.008926,-0.369009,0.214735,701824.896308-0,-309.381050,76.169033,231.239028,3.974694,-54.166874,-135.940395,-0.626711,0.260672,0.734359,NaN,NaN,NaN,NaN,NaN,NaN
10432,701824.903255,2614,0.000254,0.121284,0.805949,701824.903255-0,-187.739798,-113.750563,321.984640,3.974694,-54.166874,-135.940395,-0.383429,-0.119167,0.915850,NaN,NaN,NaN,NaN,NaN,NaN
10433,701824.911208,2614,0.003145,-0.064906,0.586606,701824.911208-0,-253.768730,-29.179092,291.779066,3.974694,-54.166874,-135.940395,-0.515487,0.049976,0.855439,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
def convertIndexToMultiIndexUsingUnderscore(labelIn):

    s = labelIn.split('_')

    if len(s) is 2:

        # otherwise, its already a scalar
        (first,second) = s
        return (first,second)

    elif len(s) == 3 :

        (first,desc,second)= s
        return (first,second)


    else:
        return (labelIn,'')

def convertIndexToMultiIndexIgnoringUnderscore(labelIn):

    if( labelIn.split('_')[-1] == 'x' or  labelIn.split('_')[-1] == 'y' or labelIn.split('_')[-1] == 'z' ):

        top = '-'.join(labelIn.split('_')[:-1])
        bottom = labelIn.split('_')[-1]
        return (top,bottom)

    else:

        return (labelIn,'')
        

In [22]:
# # Drop data below the confidence level
# filteredGazeData = rawGazeData.reset_index().drop(np.where(rawGazeData['confidence'] < gazeConfidenceThreshold )[0])

# Merge gaze and trial data
interpDF = pd.merge( rawTrialData, rawGazeData.reset_index(), on ='pupilTimestamp',how='outer',sort=True)

if(trialResults['trialType'] == 'CalibrationAssessment'):

    interpDF['frameTime'] = interpDF['frameTime'].fillna(method='ffill')
    interpDF['isHeadFixed'] = interpDF['isHeadFixed'].fillna(method='ffill')
    interpDF['currentTargetName'] = interpDF['currentTargetName'].fillna(method='ffill')
    
# Upsample trial data to the resolution of gaze data
# logger.warning('*** UPSAMPLING NON-GAZE DATA TO MATCH EYE TRACKER SAMPLING FREQUENCY ***')
interpDF = interpDF.interpolate(method='linear',downcast='infer')

# Drop matrixes from processed dataframe until I can figure out how to properly interpolate 
interpDF.drop(labels=interpDF.columns[(interpDF.columns.get_level_values(0).str.contains('4x4'))],axis=1)


# Convert to multiindex
newColList = [convertIndexToMultiIndexUsingUnderscore(c) for c in interpDF.columns[:len(rawTrialData.columns)]]
newGazeColList = [ convertIndexToMultiIndexIgnoringUnderscore(c) for c in interpDF.columns[(len(rawTrialData.columns)):] ]
newColList.extend(newGazeColList)
interpDF.columns = pd.MultiIndex.from_tuples(newColList)

# Some sanity checks 
if( len(interpDF) > ( len(rawTrialData) + len(rawGazeData)) ):
    logger.warning('len(interpDF) > ( len(rawTrialData) + len(rawGazeData))')


In [23]:
dictOut = {"rawUnityData": rawTrialData, "rawGazeData": rawGazeData, "interpolatedData": interpDF}